In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
# For tensorboard
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [ ]:
# CME and CLE parameters
concA = 10
concB = 20
k1 = 6
k2 = 1.0
k3 = 230
k4 = 1000
vol = 8

In [ ]:
# Define intensity functions
def lambda1(n):
    return concA*k1*n*(n-1)/vol
def lambda2(n):
    return k2*n*(n-1)*(n-2)/vol**2
def lambda3(n):
    return concB*k3*vol
def lambda4(n):
    return  n*k4

# Define terminal condition function
def terminalCondition(n):
    return 1.0*n

# Define drift and diffusion coefficients
def drift(x):
    return lambda1(x) - lambda2(x) + lambda3(x) - lambda4(x)

def diffusion(x):
    sigma = []
    for i in range(len(x)):
        sigma.append([np.sqrt(lambda1(x[i])), - np.sqrt(lambda2(x[i])), 
                      np.sqrt(lambda3(x[i])), -np.sqrt(lambda4(x[i]))])
    return np.array(sigma, dtype=np.float32)

def diffusiontf(x):
    sigma = [np.sqrt(lambda1(x)), - np.sqrt(lambda2(x)), 
             np.sqrt(lambda3(x)), -np.sqrt(lambda4(x))]
    return tf.convert_to_tensor(sigma)

In [ ]:
# Simulation parameters
dt = 0.0001
nreactions = 4
timesteps = 10

# Network parameters
inputSize = timesteps*(nreactions + 1) + 1
hlayer_depth = 4
hlayer_width = inputSize + 10
activation_func = tf.nn.relu 
initialLearningRate = 1e-2 #1e-3 
trainingIterations = 5000 
batchSize   = 64 #512

# Domain parameters
domain = [50,500]
outputResolution = 200


# File writing parameters
datasize = 1024*5
stride = 10
#filename = "data/schlogl_data_bckwd_v" + str(vol)"_" + str(datasize) + "_T=" + str(timesteps*dt) + ".txt"

In [ ]:
# Generate Brownian paths function
def generateBrownian(n):
    output = np.zeros([n,4])
    for i in range(n):
        output[i] = np.random.normal(0., np.sqrt(dt), nreactions)
    return output

# Propagate forward SDE function
def propagateSDE(x0, brownianPath):
    n = len(brownianPath)
    dimension = len(x0)
    trajectory = np.zeros([n, dimension])
    trajectory[0] = 1.0*x0
    for i in range(n-1):
        x = trajectory[i]
        trajectory[i+1] = x + drift(x)*dt + np.dot(diffusion(x),brownianPath[i])
        if trajectory[i+1] < 0:
            trajectory[i+1] = 0
    return trajectory

In [ ]:
# Generate network structure, see paper

# Inputs and targets placeholders for trained data
networkInput = tf.placeholder(dtype=tf.float32, shape=(None, inputSize), name='input')
networkTarget = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='target')
#networkTarget = tf.placeholder(dtype=tf.float32, shape=(None, timesteps + 1), name='target')

# Set second part of input into brownian  paths tensor
brownianPaths = (timesteps)*[None]
for k in range(timesteps):
    brownianPaths[k] = networkInput[:,timesteps + 1 + k*nreactions : timesteps + 1 + k*nreactions + nreactions]

# Create hidden layers, hidden[timestep][hlayer_depth]
hidden = [[None for i in range(hlayer_depth-1)] for j in range(timesteps)] 
for k in range(timesteps):
    # Set input of each hidden layer to X_tk (first part of input vector)
    hidden[k][0] = networkInput[:, k:k+1]
    # Connect the hidden layers for each timestep
    for i in range(hlayer_depth-2):
        hidden[k][i+1] = tf.layers.dense(hidden[k][i], hlayer_width, activation=activation_func, 
                                         name='hidden' + str(k) + '-' + str(i+1))
        
# Set output of hidden layers to be the gradients (note index doesnt match hidden layer index due to initial value)
gradients = (timesteps+1)*[None]
gradients[0] = tf.Variable(0.5, dtype=tf.float32, name='gradient0') # Initial value at X0
for k in range(timesteps):
    gradients[k+1] = tf.layers.dense(hidden[k][hlayer_depth-2], 1, activation=None, name='gradient' + str(k+1))

# Connect gradients to Euler Maruyama scheme
predictions = (timesteps + 1)*[None]
predictions[0] = tf.Variable(120.0, dtype=tf.float32, name='prediction0') # Initial value at X0
sigma = (timesteps)*[None]
noiseterm = (timesteps)*[None]
for k in range(timesteps):
    sigma[k] = tf.py_func(diffusion, [hidden[k][0]], tf.float32)
    noiseterm[k] = tf.matmul(tf.reshape(sigma[k], [-1,nreactions]) , 
                             tf.reshape(brownianPaths[k], [-1, nreactions]),
                            transpose_b = True)
    predictions[k+1] = predictions[k] + gradients[k]*noiseterm[k]

In [ ]:
# Training procedure

# Define loss function and error
loss = tf.reduce_mean(tf.square(predictions[-1] - networkTarget))
error = tf.reduce_max(tf.abs(predictions[-1] - networkTarget))

# use stochastic gradient descent with ADAM during optimization
step = tf.train.AdamOptimizer(initialLearningRate).minimize(loss)

# For tensor board
mse_summary = tf.summary.scalar('Error', error)
file_writer = tf.summary.FileWriter(logdir,	tf.get_default_graph())

In [ ]:
# Generate Training data
generateData = True
    
if generateData:
    #x0 = np.float32((domain[1] - domain[0])*np.random.rand(datasize) + domain[0])
    x0 = np.float32((domain[1]-domain[0])/2.0)
    
    num_cores = multiprocessing.cpu_count() 
    brownianTrajs = Parallel(n_jobs=num_cores, 
                             verbose = 2)(delayed(generateBrownian)(timesteps) for i in range(datasize))
    solutionsSDE = Parallel(n_jobs=num_cores, 
                            verbose = 2)(delayed(propagateSDE)(np.array([x0]),brownianTrajs[i]) for i in range(datasize))
    # No need to write to file
    #print("Writing to file ...", end="\r")
    #f = open(filename,"w")
    #for i in range(len(results)):
    #    f.write(" ".join(str(x) for x in results[i]) + "\n")
    #f.close()
    #print("Percentage finished:", 100, "%    ", end="\r")

In [ ]:
# Define input and target data
inputData = [None]*datasize
targetData = [None]*datasize
for i in range(datasize):
    flattBrownianTrajs = np.reshape(brownianTrajs[i], [-1,1])
    flattX0 = np.reshape(np.array([x0]), [-1,1])
    inputData[i] = np.concatenate((flattX0, solutionsSDE[i], flattBrownianTrajs))
    targetData[i] = terminalCondition(solutionsSDE[i][-1])   

In [ ]:
# Start Tensorflow session and initialize all network variables
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
indices =  np.random.randint(0, datasize, batchSize)
inputBatch = np.reshape(np.take(inputData, indices, axis = 0), [-1, inputSize])
targetBatch = np.reshape(np.take(np.array(targetData), indices,), [-1,1])

In [ ]:
# Run the training

# run gradient descent steps with Adam
print('\nStarted training...')
print('{:8s}\t{:8s}\t{:8s}'.format('iter', 'l2-loss', 'linf-err'))
print('{:8s}\t{:8s}\t{:8s}'.format(*(3*[8*'-'])))
for iter in range(trainingIterations):
    # generate random batch of inputs and corresponding target values
    indices =  np.random.randint(0, datasize, batchSize)
    inputBatch = np.reshape(np.take(inputData, indices, axis = 0), [-1, inputSize])
    targetBatch = np.reshape(np.take(np.array(targetData), indices), [-1,1])

    # take gradient descent step and compute loss & error
    loss_val, error_val, _ = session.run(
        [loss, error, step],
        feed_dict={networkInput: inputBatch, 
                   networkTarget: targetBatch}
    )
    if iter % 100 == 0:
        print('{:8d}\t{:1.2e}\t{:1.2e}'.format(iter, loss_val, error_val))
print('...finished training.\n')

In [ ]:
# For tensor board
file_writer.close()

In [ ]:
print(session.run(predictions[0]))

In [ ]:
x0 = np.float32((domain[1]-domain[0])/2.0)
dWs = generateBrownian(timesteps)
solutionSDE = propagateSDE(np.array([x0]), dWs)
fdWs = np.reshape(dWs, [-1,1])
fx0 = np.reshape(np.array([x0]), [-1,1])
input_test = np.concatenate((fx0, solutionSDE, fdWs))
input_test = np.reshape(input_test, [-1, inputSize])
result = session.run( gradients[0], feed_dict={networkInput: input_test})
result

In [ ]:
np.mean(targetData)

## Tests

In [ ]:
a = tf.constant([7, 8, 9, 10, 11, 12], shape=[6, 1])
func = tf.py_func(diffusion, [a], tf.float32)
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
with tf.Session() as sess:
    #a.initializer.run()
    #func.initializer.run()
    result=func.eval()
result

In [ ]:
inp = tf.constant([1,2,3,4,5,6,7,8,9,10,11,12], dtype=tf.float32)
#inp = tf.reshape(inp, shape=[-1,4])
sigma1 = tf.py_func(diffusion, [inp], tf.float32)
sigma2 = tf.reshape(sigma1, [-1,nreactions])

inp2 = tf.constant([10,20,30,40,50,60,70,80,90,100,110,120], dtype=tf.float32)
bps=3*[None]

for k in range(3):
    bps[k] = inp2[k*nreactions : k*nreactions + nreactions]
bps2 = tf.reshape(bps, [-1, nreactions])
matmul = tf.matmul(sigma2,bps2, transpose_b=True)

with tf.Session() as sess:
    result=sigma2.eval()
    result2 = bps2.eval()
    result3 = matmul.eval()
print(result,result2.transpose(),result3)